Importing required libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,ConfusionMatrixDisplay,roc_auc_score

Reading dataset

In [ ]:
df = pd.read_csv('fraud_dataset.csv')

Checking null values

In [ ]:
df.isnull().sum()

Checking number of unique values

In [ ]:
df.nunique(axis=0)

Checkiong for duplicate values

In [ ]:
df[df.duplicated()]

checking statistical distribution

In [ ]:
df.describe()

checking target columns

In [ ]:
sns.countplot(x = 'isFraud' ,data=df )
plt.title('Target Variable')

In [ ]:
categorical_columns= df.select_dtypes(include= ["object"]).columns
def categorical_bar(categorical_columns):
    for column in categorical_columns:
        plt.figure(figsize=(20,4))
        ax = plt.subplot(121)
        df[column].value_counts().plot(kind="bar")
        plt.xlabel(column)
        plt.ylabel("count of customers")
        plt.title(column)
        
        # Add percentage labels to the top of each bar
        total = sum([p.get_height() for p in ax.patches])
        for p in ax.patches:
            width = p.get_width()
            height = p.get_height()
            x, y = p.get_xy() 
            ax.annotate(f'{height/total:.1%}', (x + width/2, y + height*1.02), ha='center')

In [ ]:
categorical_bar(['type'])

In [ ]:
categorical_columns= df.select_dtypes(include= ["object"]).columns
print(categorical_columns)


BIvariate analysis of categorical with target column

In [ ]:

fig = px.histogram(df, x="type",
             color='isFraud', barmode='group',text_auto=True,
             height=400)
fig.show()

Univariate ana;ysis of numerical columns

In [ ]:
numerical_variable = df.select_dtypes(include=[np.number])

In [ ]:
def numerical_hist(columns):
    plt.style.use("ggplot")
    for column in columns:
        plt.figure(figsize=(20,4))
        ax = plt.subplot(121)
        sns.histplot(data=df, x=column,kde=True,bins=50)
        plt.title(column)

In [ ]:
numerical_hist(numerical_variable)

In [ ]:
df.info()

outlier detection

In [ ]:
def numerical_boxplot(columns):
    plt.style.use("ggplot")
    for column in columns:
        plt.figure(figsize=(20,4))
        ax = plt.subplot(121)
        sns.boxplot(data=df, x=column)
        plt.title(column)

In [ ]:
numerical_boxplot(numerical_variable)

Z score outlier detection

In [ ]:
# Z score
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(df['amount']))
outlier = np.array(np.where(z > 3))
#a = pd.DataFrame(outlier)
print(outlier.size)

IQR outlier detection

In [ ]:
lower= []
upper= []
for i in numerical_variable.columns:
    IQR= df[i].quantile(0.75) - df[i].quantile(0.25)
    lower_bound= df[i].quantile(0.25) - (1.5*IQR)
    upper_bound= df[i].quantile(0.75) + (1.5*IQR)
    
    print(i, ":", lower_bound, ",",  upper_bound)
    
    lower.append(lower_bound)
    upper.append(upper_bound)

Correlation detection

In [ ]:
corr = df.corr()# plot the heatmap
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
df.head()

'oldbalanceOrg','oldbalanceDest' these two are highly correlated wwith newballaanceorg and newcalancedest

removing highly correlated

In [ ]:
df = df.drop(['oldbalanceOrg','oldbalanceDest'],axis=1)
df

In [ ]:
corr = df.corr()# plot the heatmap
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()

%matplotlib inline 
# Generate visualizations
AV.AutoViz("fraud_dataset.csv")

In [ ]:
df.head()

In [ ]:
def encode_nominal(df,column):
    for i in column:
        dummies = pd.get_dummies(data=df[i],prefix=i)
        df = pd.concat([df,dummies],axis=1)
        df.drop([i],axis=1,inplace=True)
    return df

In [ ]:
df = encode_nominal(df,['type'])
df

model creation

In [ ]:
X = df.drop(['isFraud','nameOrig','nameDest'], axis=1)
y = df['isFraud']

TRain test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

Balancing the data

In [ ]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(sampling_strategy=0.75, random_state=101)
X_train_enn, y_train_enn = smote_enn.fit_resample(X_train, y_train)

y_train_enn.value_counts()

Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enn)
X_test_scaled = scaler.transform(X_test)

Creating fucntion to report model

In [ ]:
def report_model(model):
    y_pred = model.predict(X_test_scaled)
    print(classification_report(y_test,y_pred))
    print ("The accuracy of is : ", accuracy_score(y_test, y_pred)*100, "%")
    print ("The aurroc_auc_score of is : ", roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:, 1]))
    cm= confusion_matrix(y_test,y_pred)
    cm_plot = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
    cm_plot.plot()
    plt.show()

Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model_lg = LogisticRegression(random_state=101)

In [ ]:
model_lg.fit(X_train_scaled,y_train_enn)

In [ ]:
report_model(model_lg)

HYper parameter tuning for losgistic regression

In [ ]:
# Penalty Type
penalty = ['l1', 'l2']

# Use logarithmically spaced C values (recommended in official docs)
C = np.logspace(0, 4, 10)

In [ ]:
grid_model_lg = GridSearchCV(model_lg,param_grid={'C':C,'penalty':penalty})
grid_model_lg.fit(X_train_scaled,y_train_enn)

In [ ]:
import pickle

In [ ]:
#pickle.dump(grid_model_lg, open('grid_model_lg.pkl', 'wb'))

In [ ]:
pickled_lg = pickle.load(open('grid_model_lg.pkl', 'rb'))

In [ ]:
pickled_lg.best_params_

In [ ]:
report_model(pickled_lg)

Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=10,max_features='auto',random_state=101)

In [ ]:
model_rf.fit(X_train_scaled,y_train_enn)

In [ ]:
report_model(model_rf)

In [ ]:
n_estimators=[64,100,128,150]
max_features= [2,3,4,8]
bootstrap = [True,False]
oob_score = [True,False]

In [ ]:
param_grid = {'n_estimators':n_estimators,
             'max_features':max_features,
             'bootstrap':bootstrap,
             'oob_score':oob_score}

In [ ]:
grid_model_rf = GridSearchCV(model_rf,param_grid)
#grid_model_rf.fit(X_train_scaled,y_train_enn)

In [ ]:
#pickle.dump(grid_model_rf, open('grid_model_rf.pkl', 'wb'))

In [ ]:
pickled_rf = pickle.load(open('grid_model_rf.pkl', 'rb'))

In [ ]:
pickled_rf.best_params_

In [ ]:
report_model(pickled_rf)

XGboost

In [ ]:
from xgboost import XGBClassifier

model_xg = XGBClassifier(objective='binary:logitraw',random_state=101,tree_method="gpu_hist")

In [ ]:
model_xg.fit(X_train_scaled,y_train_enn)

In [ ]:
report_model(model_xg)

HYperparameter tuning

In [ ]:
param_grid = { 'max_depth': [3,5,7,10],
           'learning_rate': [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.2,0.5,0.7,1],
           'n_estimators': [100, 64, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

In [ ]:
grid_model_xgb = GridSearchCV(model_xg,param_grid)
#grid_model_xgb.fit(X_train_scaled,y_train_enn)

In [ ]:
#pickle.dump(grid_model_xgb, open('grid_model_xgb.pkl', 'wb'))

In [ ]:
pickled_xgb = pickle.load(open('grid_model_xgb.pkl', 'rb'))

In [ ]:
pickled_xgb.best_params_

In [ ]:
report_model(pickled_xgb)

In [ ]:
feat_xg = pd.DataFrame(data=model_xg.feature_importances_, index=X.columns, columns=["importance"]).sort_values(by='importance',ascending=False)
plt.figure(figsize=(14,6),dpi=200)
sns.barplot(data=feat_xg,x=feat_xg.index,y='importance')

plt.xticks(rotation=90);

Here for this project xgboost model performs best for us with a auc roc score of 0.9864.
